In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from tqdm import tqdm_notebook as tqdm
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import os

from dataset import * 
from model import * 
import config as config

# Hyper Param (Train)

In [46]:
TEST = True
LOG = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# (instance/batchsize)*epcho = # batch
BATCH_SIZE = 8
NUM_EPOCHS = 6
LR = 0.00001 # 1e-5

NUM_WARMUP = 100

cuda


# Load Data using dataset.py

In [4]:
train_set = FGC_Dataset(config.FGC_TRAIN, mode="train")

In [37]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [6]:
# calc pos weight for BCE
total = 0
true_cnt = 0
for instance in train_set:
    if(instance[-1] == True):
        true_cnt += 1
    total += 1
print(true_cnt)
print(total)
# to increase the value of recall in the model's criterion
pos_weight = print(torch.tensor([(total-true_cnt)/true_cnt, 1]))
print(pos_weight)
# no need to applied pos_weight = torch.tensor([total/true_cnt, total/(1-true_cnt)])?

1875
31422
tensor([15.7584,  1.0000])
None


# Build and Train Model

In [7]:
bert_encoder = BertModel.from_pretrained(config.BERT_EMBEDDING)

I0707 09:05:09.752314 139712109725504 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
I0707 09:05:09.755411 139712109725504 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_he

In [8]:
def calc_f1(pref, target):
    TP += ((pred == 1) & (target == 1)).cpu().sum()
    TN += ((pred == 0) & (target == 0)).cpu().sum()
    FN += ((pred == 0) & (target == 1)).cpu().sum()
    FP += ((pred == 1) & (target == 0)).cpu().sum()

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * recall * precision / (recall + precision)
    acc = (TP + TN) / (TP + TN + FP + FN)
    return precision, recall, F1, acc

In [9]:
model_test = BertSERModel(bert_encoder=bert_encoder, pos_weight=pos_weight)
print(next(model_test.parameters()).is_cuda)
None

False


In [10]:
#print(model_test(next(iter(trainloader)))) non deterministic output??
#print(next(iter(trainloader))[0].shape)

In [29]:
model = BertSERModel(bert_encoder=bert_encoder, pos_weight=pos_weight)
model.to(device) # means model = model.to(device)
if LOG:
    print("model in cuda?", next(model.parameters()).is_cuda)
#    from time import sleep
#    sleep(0.25)
model.train()
None

model in cuda? True


In [31]:
# saving directory
model_file_path = "baseline"

save_model_path = config.PARAM_PATH / model_file_path

if not os.path.exists(save_model_path):
    os.mkdir(save_model_path)

In [43]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [44]:
if train_set == None:
    train_set = FGC_Dataset(config.FGC_TRAIN, mode="train")
# train_set[0][0]
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [47]:
# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=LR)
num_train_optimization_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=NUM_WARMUP,
                                            num_training_steps=num_train_optimization_steps)

In [54]:
print('start training ... ')

for epoch_i in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for batch_i, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()

        batch = [data.to(device) for data in batch]

        loss = model(batch)

        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    learning_rate_scalar = scheduler.get_lr()[0]
    print('lr = %f' % learning_rate_scalar)
    print('epoch %d train_loss: %.3f' % (epoch_i, running_loss / len(dataloader_train)))

    if epoch_i % self.eval_frequency == 0:
        self.eval(dev_batches, epoch_i, trained_model_path, sp_golds, atype_golds)




  0%|          | 0/3928 [00:00<?, ?it/s]


  0%|          | 2/3928 [00:00<05:31, 11.85it/s]

start training ... 





  0%|          | 4/3928 [00:00<05:15, 12.43it/s]


  0%|          | 6/3928 [00:00<05:05, 12.83it/s]


  0%|          | 8/3928 [00:00<05:00, 13.03it/s]


  0%|          | 10/3928 [00:00<04:58, 13.15it/s]


  0%|          | 12/3928 [00:00<04:51, 13.45it/s]


  0%|          | 14/3928 [00:01<04:47, 13.61it/s]


  0%|          | 16/3928 [00:01<04:44, 13.76it/s]


  0%|          | 18/3928 [00:01<04:47, 13.58it/s]


  1%|          | 20/3928 [00:01<04:52, 13.34it/s]


  1%|          | 22/3928 [00:01<04:48, 13.53it/s]


  1%|          | 24/3928 [00:01<04:44, 13.73it/s]


  1%|          | 26/3928 [00:01<04:41, 13.86it/s]


  1%|          | 28/3928 [00:02<04:39, 13.94it/s]


  1%|          | 30/3928 [00:02<04:37, 14.03it/s]


  1%|          | 32/3928 [00:02<04:37, 14.03it/s]


  1%|          | 34/3928 [00:02<04:39, 13.92it/s]


  1%|          | 36/3928 [00:02<04:39, 13.90it/s]


  1%|          | 38/3928 [00:02<04:38, 13.96it/s]


  1%|          | 40/3928 [00:02<04:37, 14.03it/s]


  1%|       

RuntimeError: CUDA error: device-side assert triggered